In [1]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=9743d98eba2eabf4b95a36689d734e5a80c8296925e366ed41fd85a3ad332d7e
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [2]:
with open(encoding='utf-8', file='/kaggle/input/american-psycho-x-nlp/AMERICAN_PSYCHO.txt', mode='r') as input_fp:
    text = input_fp.read()
len(text)

123635

In [3]:
text.count('\n')

5457

In [4]:
import pandas as pd
from nltk.tokenize import sent_tokenize

df = pd.DataFrame(data={'text': sent_tokenize(text=text.replace('\n', ' '), language='english')})
df['text'] = df['text'].apply(func=lambda x: ' '.join(x.split()))
df = df.drop_duplicates(ignore_index=True)
df['char count'] = df['text'].str.len()
df['word count'] = df['text'].apply(func=lambda x: len(x.split()))
print('one-token sentences: {}'.format(df[df['word count'] == 1]['text'].values.tolist()))
print('two-token sentences: {}'.format(df[df['word count'] == 2]['text'].values.tolist()))
df = df[df['word count'] > 1] # let's keep the 2-word lines and drop the one-word ones because they often include character names
df['text60'] = df['text'].str[:60]
df = df.sample(frac=1.0, random_state=2023)
print('we have {} sentences in our corpus.'.format(len(df)))

one-token sentences: ['1.', 'Clarify.', '3.', '4.', 'Steroids...', '10.', 'Late?', 'Sorry.', 'When?', 'Thanks.', 'No.', 'Yes?', 'Godiva.', 'Photographers.', 'Evelyn?', '17.', 'Pause.', 'Soho.', '19.', 'Price?', 'Hello.', 'Bleach-ee.', 'One.', '23.', 'Understand?', 'Patrick?', 'Well.', 'Crahapple.', '24.', '25.', 'Yeah.', 'Okay.', 'Do.', 'Pumpkin?', 'dinner?', 'Nice?', 'Eight?', 'Dorsia.', 'Straight.', '29.', 'Yes.', 'So?', 'Why?', 'Huh?', 'Happy?', 'Squash?', 'Dorsia?', 'Nice.', 'Al.', 'Al...', '43.', 'What?', '47.', 'STREET/INT.', 'Couples.', 'Blonde.', '50.', '51.', 'Owen!', '58.', '59.', 'Salon.', '60.', '62.', 'Dallas?', 'Paris?', 'London.', 'Who?', 'Silence.', '66.', 'see...Yes.', '67.', '68.', 'Shoot.', 'KIMBALL.', 'Nuprin?', '71.', "Harry's.", 'Fluties.', 'Indochine.', "Nell's.", '72.', 'Hmm.', '74.', 'Nothing.', '75.', 'Maybe.', 'Don.', '79.', '81.', 'DAISY.', 'This.', 'I...am...leaving!', 'Rehab?', 'Fuckheads!', '83.', 'Jean?', '85.', '86.', 'Perfect.', 'Sorbet?', '90.', 'Bye.

In [5]:
from plotly.express import scatter
scatter(data_frame=df.sort_values(ascending=True, by='word count').reset_index(), y=['word count', 'char count'] )

In [6]:
scatter(data_frame=df, x='word count', y='char count', hover_name='text60', trendline='ols', log_x=True, log_y=True)

In [7]:
from arrow import now
from sentence_transformers import SentenceTransformer
MODELS = ['all-MiniLM-L12-v2', 'all-MiniLM-L6-v2', 'all-mpnet-base-v2', 'multi-qa-mpnet-base-dot-v1', ]
print(sorted(MODELS))
# todo look here: https://www.sbert.net/docs/pretrained_models.html

MODEL = MODELS[0] 
model_start = now()
model = SentenceTransformer(model_name_or_path=MODEL,)
print('model max sequence length: {}'.format(model.max_seq_length))
embeddings = model.encode(df['text'].values.tolist())
print('model time: {}'.format(now() - model_start))

['all-MiniLM-L12-v2', 'all-MiniLM-L6-v2', 'all-mpnet-base-v2', 'multi-qa-mpnet-base-dot-v1']


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/573 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

model max sequence length: 128


Batches:   0%|          | 0/82 [00:00<?, ?it/s]

model time: 0:00:23.874653


In [8]:
from sentence_transformers import util

similarity_start = now()
similarities = util.cos_sim(embeddings, embeddings)
combinations = sorted([[similarities[i][j], i, j] for i in range(len(similarities)) for j in range(i+1, len(similarities))], key=lambda x: x[0], reverse=True)
print('similarity time: {}'.format(now() - similarity_start))

similarity time: 0:02:32.454809


In [9]:
for score, i, j in combinations:
    if score > 0.96:
        print('similarity: {:.4f} indices: {} {} left: {} right: {}'.format(similarities[i][j], i, j, df['text60'].values[i], df['text60'].values[j], ))

similarity: 1.0000 indices: 270 2599 left: Put down your weapon. right: PUT DOWN YOUR WEAPON.
similarity: 0.9974 indices: 583 872 left: HOMELESS MAN I'm so hungry. right: HOMELESS MAN I’m so hungry.
similarity: 0.9928 indices: 609 2274 left: He pauses in the hallway and tucks some long blonde hair bac right: He pauses in the’ hallway and tucks some long blonde hair ba
similarity: 0.9908 indices: 137 2316 left: BATEMAN Oh, right. right: BATEMAN Oh right.
similarity: 0.9836 indices: 1771 1833 left: BATEMAN Thanks. right: BATEMAN Thank you.
similarity: 0.9779 indices: 1494 2053 left: BATEMAN Yes. right: BATEMAN Yeah.
similarity: 0.9777 indices: 1248 1494 left: BATEMAN Oh yes. right: BATEMAN Yes.
similarity: 0.9757 indices: 718 851 left: BATEMAN I'm leaving now. right: BATEMAN I'm just leaving.
similarity: 0.9735 indices: 1260 1494 left: BATEMAN Yes? right: BATEMAN Yes.
similarity: 0.9735 indices: 13 2349 left: BATEMAN Listen, wait. right: BATEMAN Listen.
similarity: 0.9722 indices: 1244 1

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
from plotly.colors import qualitative

TOKENIZER = RegexpTokenizer(r'\w+')

def find_term(arg:str, terms: list):
    lower = TOKENIZER.tokenize(text=arg.lower())
    for term in terms:
        if term in lower:
            return term
    return 'none'

DISCARD = {'like', 'don', 'll', 've', 'know', 'want', 'really', 'think', 'just', 'yes', }
vectorizer = TfidfVectorizer(ngram_range=(1, 1), stop_words='english', max_df=0.5, min_df=0.001)
fit_result = vectorizer.fit_transform(raw_documents=df['text'].values)
word_weight_df = pd.DataFrame(data={'word': vectorizer.get_feature_names_out(), 'weight': fit_result.toarray().sum(axis=0),}).sort_values(ascending=False, by='weight')
top_terms = [word for word in word_weight_df.head(n=25)['word'].values.tolist() if word not in DISCARD]
top_terms = top_terms + ['patten', 'hamilton']
print('top terms: {}'.format(sorted(top_terms)))

top terms: ['bateman', 'christie', 'courtney', 'elizabeth', 'evelyn', 'hamilton', 'int', 'jean', 'kimball', 'man', 'mcdermott', 'oh', 'owen', 'patrick', 'patten', 'paul', 'price']


In [11]:
from plotly.express import bar
from umap import UMAP

umap_model = UMAP(n_components=2, random_state=2023, verbose=True, n_jobs=1)
df[['u0', 'u1']] = umap_model.fit_transform(X=embeddings)
plot_df = df.copy()
plot_df['term'] = plot_df['text'].apply(func=find_term, args=(top_terms, ))
bar(data_frame=plot_df['term'].value_counts().to_frame(), y='count', log_y=True)

UMAP(n_jobs=1, random_state=2023, verbose=True)
Fri Dec 22 17:30:23 2023 Construct fuzzy simplicial set
Fri Dec 22 17:30:27 2023 Finding Nearest Neighbors
Fri Dec 22 17:30:30 2023 Finished Nearest Neighbor Search
Fri Dec 22 17:30:32 2023 Construct embedding


Epochs completed:   0%|            0/500 [00:00]

	completed  0  /  500 epochs
	completed  50  /  500 epochs
	completed  100  /  500 epochs
	completed  150  /  500 epochs
	completed  200  /  500 epochs
	completed  250  /  500 epochs
	completed  300  /  500 epochs
	completed  350  /  500 epochs
	completed  400  /  500 epochs
	completed  450  /  500 epochs
Fri Dec 22 17:30:36 2023 Finished embedding


In [12]:
from plotly.express import scatter
scatter(data_frame=plot_df, x='u0', y='u1', hover_name='text60', color='term', color_discrete_sequence=qualitative.Light24, height=800)

The vectorizer plus the LLM give us a representation of the script that will identify the major characters pretty well.